In [ ]:
from autumn.projects.sm_covid2.common_school.project_maker import get_school_project_parameter_set
import pandas as pd
from copy import copy
from pathlib import Path

In [ ]:
model_params = get_school_project_parameter_set(
    iso3 = "FRA",
    first_date_with_death = 0,
    sero_age_min = 0,
    sero_age_max = 100,
    analysis='main'
).baseline
age_params = model_params['age_stratification']

In [ ]:
df = pd.DataFrame()
df['ifr'] = pd.Series(age_params['ifr']['values'])
df['susceptibility'] = pd.Series(age_params['susceptibility'])
df['prop_symptomatic'] = pd.Series(age_params['prop_symptomatic'])
df['prop_hospital'] = pd.Series(age_params['prop_hospital']['values'])

In [ ]:
# fill missing values for 80+ category
for column in ("susceptibility", "prop_symptomatic", "prop_hospital"):
    df[column].loc[80] = df[column].loc[75]

In [ ]:
# new column for age group
def get_agegroup_str(idx):
    if idx < 80:
        return f"{idx}-{idx + 4}"
    else:
        return "80 and above"

df['agegroup'] = [get_agegroup_str(idx) for idx in df.index]

In [ ]:
pd.set_option('display.max_colwidth', 2)
tex_table = df.to_latex(
    columns=['agegroup', 'susceptibility', 'prop_symptomatic', 'prop_hospital', 'ifr'],
    header=['Age group', "Rel. suscepitbility to infection (ref. 15-69 y.o.) \cite{zhang2020a}", "Proportion symptomatic \cite{sah2021}", "Proportion of symptomatic patients hospitalised \cite{rivm2020}", "Infection fatality rate \cite{odriscoll2021}"],
    index=False,
    label="agespec_table",
    caption="Age-specific parameters for wild-type COVID-19"
)

In [ ]:
tex_table = tex_table.replace("{lrrrr}", "{p{2cm} p{3cm} p{3cm} p{3cm} p{3cm}}")

In [ ]:
# Handle citations
tex_table = tex_table.replace("\\textbackslash cite\\", r"\cite").replace("\\}", "}")

In [ ]:
out_path = Path.home() / "Models"/ "AuTuMN_new" / "docs" / "tex" / "tex_descriptions" / "projects" / "sm_covid" / "agespec_table.tex"

In [ ]:
with out_path.open("w") as f:
    f.write(tex_table)